# Battery Calculations

This is a second attempt to calculate battery requirements for a custom made satellite tracker that needs to last 2 weeks.

The first attempt is located [here](https://github.com/ssfivy/blackbox-hw/wiki)

## Constraints

### Power supply constraints - Volume & Weight
The entire thing needs to sit inside a sealed box. We have the external box volume but not internal dimensions, so lets approximate it and say the box is 5mm thick (we don't want to cut things this close anyway)

The entire thing also has an upper limit of weight including contents, which is 5kg. We don't know the weight of the box but for now let us assume it is 500gr.

In [78]:
# minus 5 mm on both sides in each dimension
box_volume_mm3 = (200-10) * (150-10) * (100-10 )
# minus 5000gr box weight
box_weight_kg = 5 - 0.500

### Power supply constraints - Battery technology
Our major limitation is the energy density.
We have two options: Rechargeable vs non-rechargeable batteries.
Prime candidate for rechargeable is the Panasonic NCR18650B cells - sturdy cells from trustworthy manufacturers which is not likely to catch on fire.
For non-rechargeable, our candidate is the Energiser Ultimate Lithium - high energy density with good temperature range.

Since measuring battery capacity is another whole topic on its own, we will simply use a rough approximation of using mAh rating x nominal voltage to get energy capacity.

Sources:
- http://www.batteryspace.com/prod-specs/NCR18650B.pdf
- https://data.energizer.com/pdfs/l91.pdf

In [79]:
import math

cell = {'panasonic':{}, 'energizer':{}}
cell['panasonic']['weight_kg'] = 0.0485
cell['panasonic']['volume_mm3'] = math.pi*((18.5/2)**2) * 65.3
cell['panasonic']['energy_Ah'] = 3.250
cell['panasonic']['voltage_nominal_V'] = 3.6

cell['panasonic']['energy_J'] =  cell['panasonic']['energy_Ah'] *  cell['panasonic']['voltage_nominal_V'] * 3600
cell['panasonic']['energydensity_WhperKg'] = 243 # from datasheet
cell['panasonic']['energydensity_Whpermm3'] = 676 / 1000000 # from datasheet / 1000000 mm3 in a liter

cell['energizer']['weight_kg'] = 0.015
cell['energizer']['volume_mm3'] = math.pi*((14.5/2)**2) * 50.5 # worst case
#cell['energizer']['volume_mm3'] = math.pi*((14.5/2)**2) * 49 + math.pi*((5.5/2)**2) * 1 # more accurate
cell['energizer']['energy_Ah'] = 3.500
cell['energizer']['voltage_nominal_V'] = 1.5

cell['energizer']['energy_J'] =  cell['energizer']['energy_Ah'] *  cell['energizer']['voltage_nominal_V'] * 3600
cell['energizer']['energydensity_WhperKg'] =  cell['energizer']['energy_Ah'] *  cell['energizer']['voltage_nominal_V'] / cell['energizer']['weight_kg']
cell['energizer']['energydensity_Whpermm3'] = cell['energizer']['energy_Ah'] *  cell['energizer']['voltage_nominal_V'] / cell['energizer']['volume_mm3']

def print_energy_calcs(cell_type):
    print('{} : Energy per cell (Joules) : {:.2f}'.format(
        cell_type, cell[cell_type]['energy_J']))
    print('{} : Energy density (Wh/kg) : {:.2f}'.format(
        cell_type, cell[cell_type]['energydensity_WhperKg'])) 
    print('{} : Energy density (Wh/mm3) : {:.7f}'.format(
        cell_type, cell[cell_type]['energydensity_Whpermm3'])) 
    print('')

print_energy_calcs('panasonic')
print_energy_calcs('energizer')

panasonic : Energy per cell (Joules) : 42120.00
panasonic : Energy density (Wh/kg) : 243.00
panasonic : Energy density (Wh/mm3) : 0.0006760

energizer : Energy per cell (Joules) : 18900.00
energizer : Energy density (Wh/kg) : 350.00
energizer : Energy density (Wh/mm3) : 0.0006296



### Power usage constraints - Time
The event will last one full week, however the devices will be installed in scrutineering in the week before. It would be nice to see the data flowing in in preparation for the event beforehand too. So from Monday week 1 to Saturday week 2 is a total of 13 days, and our power must be available all this time.

We can optimise by not being active at night when nothing happened, effectively curring power usage in half.

In [80]:
total_uptime_secs = 13 * 86400 # days x seconds/day
total_uptime_secs_optimised = total_uptime_secs / 2

### Power usage constraints - Satellite Modems
There are several satellite modems available. They don't have ultra-detailed power consumption graphs, so we are missing a particularly important value: the length of time those devices stay in transmit mode, with the extremely high power consumption. We will simply pick a reporesentative device from each provider and use some reasonable averaging / guesstimate for this initial calculation.

#### Globalstar
#### Inmarsat
#### Iridium
#### Thuraya

In [81]:
# TODO FIXME

### Power usage constraints - GPS receiver

Like the sat modems, there's no power consumption graphs, and there's too many vendors out there. So let us pick one that we can get good information / supply for and use that as a representative sample.

- uBlox Neo-M9N  https://www.sparkfun.com/products/15712
- alternative: the Adafruit Ultimate GPs seems to have better power figures

In [82]:
gps_acqusition_power_W = 0.100 * 3 # 100mA * 3V
gps_runtime_power_W = 0.030 * 3 # 30mA x 3V
# TODO FIXME: How long is acqusition and how long is runtime?

### Power usage constraints -  Processors
We can either go Linux-based or microcontroller-based here. Linux-based will consume more power but allows more extensive features in device.Microcontroller-based is likely more robust.
We pick a raspi zero w for Linux because someone has run the numbers and a STM32L451 for micro because I am familiar with ST's stuff.

For microcontroller, we take the power consumption of running at 80MHz with PLL and no peripherals. It should be usable as a rough figure for battery estimation considering we only need a 2-week uptime instead of years, especially if we then run it at lower clocks and bypass PLL to compensate for enabled peripherals.

- https://www.jeffgeerling.com/blogs/jeff-geerling/raspberry-pi-zero-power
- https://www.st.com/content/st_com/en/products/microcontrollers-microprocessors/stm32-32-bit-arm-cortex-mcus/stm32-ultra-low-power-mcus/stm32l4-series/stm32l4x1/stm32l451re.html#resource

In [83]:
raspi0_linux_power_W = 0.080 * 5 # 80mA * 5V
stm32_micro_power_W = 0.010 * 3.3 # 10mA * 3.3V

### Power usage constraints - Miscellaneous
Miscellaneous power usage such as extra chips, power supply inneficiency, etc
Numbers pulled out of thin air by me.

In [84]:
# TODO FIXME

## Calculations

### Power usage
We have several constraints with several options each. Since creating a full combination matrix can get really messy, let's start off with best-case and worst-case scenario.

In [85]:
case = {'best':{}, 'worst':{}}

# Best case: Optimised time, most efficient satmodems, few acqusition time, stm32 micro.

case['best']['average_power_W'] = gps_runtime_power_W + stm32_micro_power_W
case['best']['total_energy_J'] = case['best']['average_power_W'] * total_uptime_secs_optimised
print(case['best']['total_energy_J'])

# Worst case: Full time, powerhog modems, long acqusition time, Linux processor
case['worst']['average_power_W'] = gps_runtime_power_W + raspi0_linux_power_W
case['worst']['total_energy_J'] = case['worst']['average_power_W'] * total_uptime_secs
print(case['worst']['total_energy_J'])

69076.8
550368.0


### Power storage
Again, we have several constraints with several options each, so let's go with a worst and best case scenario.
We define functions to calculate the number of cells, volume, and weight needed for a given amount of energy consumption.

In [86]:
def calc_battery(energy_J, battery_type):
    cell_count = math.ceil(energy_J / cell[battery_type]['energy_J'])
    cell_volume = cell_count * cell[battery_type]['volume_mm3']
    cell_weight = cell_count * cell[battery_type]['weight_kg']
    print('With {} batteries, we need {} cells that occupies {:.2f} mm3 and {:.2f} kg'.format(
        battery_type, cell_count, cell_volume, cell_weight))
    print('This takes {:.2f} % of volume and {:.2f} % of weight capacity'.format(
        cell_volume / box_volume_mm3, cell_weight / box_weight_kg))
    print('note the figures above does not count electronics, wiring, etc\n')

calc_battery(case['best']['total_energy_J'], 'panasonic')
calc_battery(case['best']['total_energy_J'], 'energizer')
calc_battery(case['worst']['total_energy_J'], 'panasonic')
calc_battery(case['worst']['total_energy_J'], 'energizer')

With panasonic batteries, we need 2 cells that occupies 35105.61 mm3 and 0.10 kg
This takes 0.01 % of volume and 0.02 % of weight capacity
note the figures above does not count electronics, wiring, etc

With energizer batteries, we need 4 cells that occupies 33356.25 mm3 and 0.06 kg
This takes 0.01 % of volume and 0.01 % of weight capacity
note the figures above does not count electronics, wiring, etc

With panasonic batteries, we need 14 cells that occupies 245739.27 mm3 and 0.68 kg
This takes 0.10 % of volume and 0.15 % of weight capacity
note the figures above does not count electronics, wiring, etc

With energizer batteries, we need 30 cells that occupies 250171.90 mm3 and 0.45 kg
This takes 0.10 % of volume and 0.10 % of weight capacity
note the figures above does not count electronics, wiring, etc

